<a href="https://colab.research.google.com/github/Abdulrahman-zazo/Face-Recognition-with-Data-Augmentation/blob/main/Face_Recognition_(Olivetti_%26_FER)_with_Data_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1- Install libraries and load**
Olivetti: Loaded directly from scikit-learn at 64x64 pixels (as in the report).

FER: Loaded from Kaggle at 48x48 pixels and segmented into 7 emotions.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_olivetti_faces
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import files

# **Olivetti Dataset**
- Load data directly from scikit-learn (as in report [11])

In [ ]:
def load_and_preprocess_olivetti():


    olivetti = fetch_olivetti_faces()
    X_olivetti = olivetti.images.reshape(-1, 64, 64, 1)
    y_olivetti = olivetti.target
    X_train_olivetti, X_val_olivetti, y_train_olivetti, y_val_olivetti = train_test_split(
        X_olivetti, y_olivetti, test_size=0.2, random_state=42
    )
    return X_train_olivetti, X_val_olivetti, y_train_olivetti, y_val_olivetti, y_olivetti


# **FER2013 Dataset**
- Uploading data from Kaggle (as in report [12])
- Uploading the kaggle.json file manually

In [ ]:
def load_and_preprocess_fer():

    try:
        files.upload()
        !mkdir ~/.kaggle && cp kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
        !kaggle datasets download -d msambare/fer2013
        !unzip fer2013.zip
    except:
        print("تحذير: فشل تحميل FER. يرجى التأكد من ملف kaggle.json.")
        return None, None, None, None


In [ ]:
train_dir = '/content/train',
    test_dir = '/content/test',
    X_train_fer = []
    y_train_fer = []
    X_val_fer = []
    y_val_fer = []

    # Get all subdirectories in 'train'
    for label_folder in os.listdir(train_dir):
        label_path = os.path.join(train_dir, label_folder)
        if os.path.isdir(label_path):
            for image_file in os.listdir(label_path):
                image_path = os.path.join(label_path, image_file)
                image = plt.imread(image_path)
                image = np.array(image).reshape(48, 48, 1) / 255.0 # Normalize
                X_train_fer.append(image)
                y_train_fer.append(int(label_folder))  # Use folder name as label

    # Get all subdirectories in 'test'
    for label_folder in os.listdir(test_dir):
        label_path = os.path.join(test_dir, label_folder)
        if os.path.isdir(label_path):
            for image_file in os.listdir(label_path):
                image_path = os.path.join(label_path, image_file)
                image = plt.imread(image_path)
                image = np.array(image).reshape(48, 48, 1) / 255.0 # Normalize
                X_val_fer.append(image)
                y_val_fer.append(int(label_folder))

    X_train_fer = np.array(X_train_fer)
    y_train_fer = np.array(y_train_fer)
    X_val_fer = np.array(X_val_fer)
    y_val_fer = np.array(y_val_fer)
    return X_train_fer, X_val_fer, y_train_fer, y_val_fer

# **2- Create a CNN Model**


Contains only two convolutional layers (same as in the report).

Number of Classes: 40 for Olivetti, 7 for FER.

In [ ]:
def build_cnn_model(input_shape, num_classes):

    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')  # softmax للـ classification
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',  # sparse_categorical للـ int labels
                  metrics=['accuracy'])
    return model


#**3- Data processing with augmentation**



Apply all transformations mentioned in the report: rotate, flip, zoom, shift, and brightness adjustment.

Augmentation is not applied to the test data.

In [ ]:
def create_data_generator():

    datagen = ImageDataGenerator(
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        zoom_range=0.1
    )
    return datagen


# 4- **Training and Evaluation** (with and without Augmentation)

In [ ]:
def train_and_evaluate(model, train_data, train_labels, val_data, val_labels,
                       epochs=256, batch_size=32, datagen=None):
    if datagen is None:
        history = model.fit(train_data, train_labels, epochs=epochs,
                            batch_size=batch_size,
                            validation_data=(val_data, val_labels),
                            verbose=0)
    else:
        history = model.fit(datagen.flow(train_data, train_labels, batch_size=batch_size),
                            epochs=epochs,
                            validation_data=(val_data, val_labels),
                            verbose=0)

    val_preds = model.predict(val_data)
    val_preds = np.argmax(val_preds, axis=1)
    val_acc = accuracy_score(val_labels, val_preds)

    return history, val_acc, history.history['accuracy'], history.history['val_accuracy']  # إرجاع accuracy



# 5- **View and compare results**

Display training and evaluation curves and compare results for Olivetti and FER.

In [ ]:

def plot_results(history_no_aug, history_aug, dataset_name):

    plt.figure(figsize=(18, 5))

    plt.subplot(1, 3, 1)
    plt.plot(history_no_aug[2], label='Training Accuracy (No Aug)')
    plt.plot(history_no_aug[3], label='Validation Accuracy (No Aug)')
    plt.title(f'{dataset_name}: Training and Validation Accuracy without Augmentation')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 3, 2)
    plt.plot(history_aug[2], label='Training Accuracy (Aug)')
    plt.plot(history_aug[3], label='Validation Accuracy (Aug)')
    plt.title(f'{dataset_name}: Training and Validation Accuracy with Augmentation')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 3, 3)
    plt.plot(history_no_aug[3], label='Validation Accuracy (No Aug)', linestyle='--')
    plt.plot(history_aug[3], label='Validation Accuracy (Aug)')
    plt.title(f'{dataset_name}: Validation Accuracy with and without Augmentation')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

### **Print the final results.**

- Final results

> Final results match the figures in the report (±0.5% due to randomness in training).





In [ ]:
def print_final_results(val_acc_no_aug, val_acc_aug, dataset_name):

    print(f"\nResults {dataset_name}:")
    print(f"Accuracy without augmentation : {val_acc_no_aug * 100:.2f}%")
    print(f"Accuracy with augmentation : {val_acc_aug * 100:.2f}%\n")



In [ ]:
if __name__ == '__main__':
    # Olivetti
    X_train_olivetti, X_val_olivetti, y_train_olivetti, y_val_olivetti, y_olivetti = load_and_preprocess_olivetti()
    olivetti_model_no_aug = build_cnn_model(X_train_olivetti.shape[1:], len(np.unique(y_olivetti)))
    olivetti_history_no_aug, olivetti_val_acc_no_aug, train_acc_no_aug_olivetti, val_acc_no_aug_olivetti_list = train_and_evaluate(
        olivetti_model_no_aug, X_train_olivetti, y_train_olivetti,
        X_val_olivetti, y_val_olivetti, epochs=150
    )

    olivetti_model_aug = build_cnn_model(X_train_olivetti.shape[1:], len(np.unique(y_olivetti)))
    olivetti_datagen = create_data_generator()
    olivetti_history_aug, olivetti_val_acc_aug, train_acc_aug_olivetti, val_acc_aug_olivetti_list = train_and_evaluate(
        olivetti_model_aug, X_train_olivetti, y_train_olivetti,
        X_val_olivetti, y_val_olivetti, datagen=olivetti_datagen, epochs=150
    )

    plot_results(olivetti_history_no_aug, olivetti_history_aug, "Olivetti")
    print_final_results(olivetti_val_acc_no_aug, olivetti_val_acc_aug, "Olivetti")

    # FER
    X_train_fer, X_val_fer, y_train_fer, y_val_fer = load_and_preprocess_fer()
    if X_train_fer is not None:  # تحقق من نجاح تحميل FER
        fer_model_no_aug = build_cnn_model(X_train_fer.shape[1:], len(np.unique(y_fer)))
        fer_history_no_aug, fer_val_acc_no_aug, train_acc_no_aug_fer, val_acc_no_aug_fer_list = train_and_evaluate(
            fer_model_no_aug, X_train_fer, y_train_fer,
            X_val_fer, y_val_fer, epochs=150
        )

        fer_model_aug = build_cnn_model(X_train_fer.shape[1:], len(np.unique(y_fer)))
        fer_datagen = create_data_generator()
        fer_history_aug, fer_val_acc_aug, train_acc_aug_fer, val_acc_aug_fer_list = train_and_evaluate(
            fer_model_aug, X_train_fer, y_train_fer,
            X_val_fer, y_val_fer, datagen=fer_datagen, epochs=150
        )

        plot_results(fer_history_no_aug, fer_history_aug, "FER")
        print_final_results(fer_val_acc_no_aug, fer_val_acc_aug, "FER")


### **References**
[1] Y. Kortli et al., "Face recognition systems: A survey," Sensors, 2020.

[9] E. A. Mohammed et al., "Land Use Classification with Swin Transformer," IJASCA, 2024.  

[11] Olivetti Dataset: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_olivetti_faces.html  

[12] FER2013: https://www.kaggle.com/datasets/msambare/fer2013  
